In [1]:
## load necessary libraries
%matplotlib inline
#%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider, Button, RadioButtons
import os,time,subprocess,glob
from PIL import Image
import cripser as cr
import tcripser as tcr
import persim
import skimage
from skimage import io
from skimage.transform import rotate
from skimage.data import cells3d
from skimage.filters import threshold_otsu
from scipy.ndimage import distance_transform_edt,convolve
import os, re
from tkinter import filedialog
import tkinter as tk


C:\Users\jeeva\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
def dt(img,radius=15,signed=False):
#    bw_img = (img >= rank.otsu(img, disk(radius)))
    bw_img = (img >= threshold_otsu(img))
    dt_img = distance_transform_edt(bw_img)
    if signed:
        dt_img -= distance_transform_edt(~bw_img)
    return(dt_img)

def explore_slices(data, cmap="gray"):
    from ipywidgets import interact
    N = data.shape[-1]
    @interact(plane=(0, N - 1))
    def display_slice(plane=N//2):
        fig, ax = plt.subplots(figsize=(20, 5))
        plt.imshow(data[:,:,plane],cmap=cmap)
        plt.show()
    return display_slice


In [ ]:
def load_image():
    root = tk.Tk()
    root.withdraw() # Hide the root window

    # Let the user choose the file
    file_path = filedialog.askopenfilename()

    return file_path

# Load image data interactively
imgfile = load_image()

# load image data
if os.path.isfile(imgfile):
    img2d = np.array(Image.open(imgfile).convert('L'))
else: # if file is not found, load the demo img from skimage
    img2d = skimage.data.camera()
    io.imsave(imgfile, img2d)

## apply distance transform
img2d_dt = dt(img2d)

fig,axs = plt.subplots(1,2)
axs[0].imshow(img2d)
axs[1].imshow(img2d_dt)
print(img2d.shape, img2d.dtype, f"{np.min(img2d)}--{np.max(img2d)}")
plt.show()

In [ ]:
start = time.time()
pd = cr.computePH(img2d)
pds = [pd[pd[:,0] == i] for i in range(3)]
print("Betti numbers: ",[len(pds[i]) for i in range(len(pds))])

# compute PH for the T-construction of the original image (pixel value filtration)
pdt = tcr.computePH(img2d)
pdst = [pdt[pdt[:,0] == i] for i in range(3)]
print("Betti numbers: ",[len(pdst[i]) for i in range(len(pdst))])

print ("elapsed_time:{} sec".format(time.time() - start))
## plot persistent diagram using persim
fig,axs = plt.subplots(1,2)
persim.plot_diagrams([p[:,1:3] for p in pds], ax=axs[0], title='V-construction')
persim.plot_diagrams([p[:,1:3] for p in pdst], ax=axs[1], title='T-construction')
